In [1]:
import torch
import os
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_curve

In [2]:
def get_threshold(tpr, threshold):
    target = 10**(-2)
    for i in range(len(tpr)):
        if tpr[i] <= target and tpr[i+1] > target:
            return threshold[i]

In [3]:
from shutil import copy2

In [4]:
races = ['African','Asian','Caucasian','Indian']
genders = ['Man','Woman']
labels = []
dist = []
for race in races:
    for gender in genders:
        with open(f"./lmks/{race}_{gender}_lmks.csv",'r') as f:
            lines = f.readlines()
        for line in lines:
            arr = line.strip().split(',')
            labels.append(int(arr[3]))
            dist.append(float(arr[2]))

In [5]:
fpr , tpr ,thresholds = roc_curve(labels,dist)
threshold = get_threshold(tpr,thresholds)
threshold

0.9695320129394531

In [6]:
races = ['African','Asian','Caucasian','Indian']
genders = ['Man','Woman']
accuracies = {}
for race in races:
    for gender in genders:
        accuracies[f"{race}_{gender}"] = {'TP':0,'TN':0,'FP':0,'FN':0}
        with open(f"./lmks/{race}_{gender}_lmks.csv",'r') as f:
            lines = f.readlines()
        dist = []
        labels = []
        for line in lines:
            arr = line.strip().split(',')
            dist.append(float(arr[2]))
            labels.append(int(arr[3]))
            if float(arr[2]) < threshold:
                with open(f"./results/{race}_{gender}_results.csv",'a') as f:
                    f.write(f"{arr[0]},{arr[1]},0,{arr[3]}\n")
                if int(arr[3]) == 1:
                    accuracies[f"{race}_{gender}"]['TN'] += 1
                else:
                    accuracies[f"{race}_{gender}"]['FN'] += 1
            else:
                with open(f"./results/{race}_{gender}_results.csv",'a') as f:
                    f.write(f"{arr[0]},{arr[1]},1,{arr[3]}\n")
                if int(arr[3]) == 1:
                    accuracies[f"{race}_{gender}"]['TP'] += 1
                else:
                    accuracies[f"{race}_{gender}"]['FP'] += 1
        fpr, tpr, thresholds = roc_curve(labels,dist)
        fpr = np.array(fpr)
        tpr = np.array(tpr)
        with open(f"./roc/{race}_{gender}_roc.npy",'wb') as f:
            np.save(f,fpr)
            np.save(f,tpr)

In [7]:
print(accuracies)

{'African_Man': {'TP': 3, 'TN': 2936, 'FP': 2, 'FN': 2938}, 'African_Woman': {'TP': 0, 'TN': 61, 'FP': 0, 'FN': 60}, 'Asian_Man': {'TP': 17, 'TN': 2111, 'FP': 6, 'FN': 2125}, 'Asian_Woman': {'TP': 11, 'TN': 861, 'FP': 6, 'FN': 863}, 'Caucasian_Man': {'TP': 29, 'TN': 2224, 'FP': 7, 'FN': 2245}, 'Caucasian_Woman': {'TP': 31, 'TN': 716, 'FP': 7, 'FN': 741}, 'Indian_Man': {'TP': 12, 'TN': 2303, 'FP': 8, 'FN': 2309}, 'Indian_Woman': {'TP': 16, 'TN': 669, 'FP': 3, 'FN': 679}}


In [8]:
gar = {}
far = {}
scores = {}
for race in races:
    for gender in genders:
        tp = accuracies[f"{race}_{gender}"]['TP']
        tn = accuracies[f"{race}_{gender}"]['TN']
        fp = accuracies[f"{race}_{gender}"]['FP']
        fn = accuracies[f"{race}_{gender}"]['FN']
        gar[f"{race}_{gender}"] = 100*tp/(tn+tp)
        far[f"{race}_{gender}"] = 100*fp/(fn+fp)
        scores[f"{race}_{gender}"] = 100*(fn+tp)/(fn+fp+tn+tp)

In [9]:
gar

{'African_Man': 0.10207553589656346,
 'African_Woman': 0.0,
 'Asian_Man': 0.7988721804511278,
 'Asian_Woman': 1.261467889908257,
 'Caucasian_Man': 1.2871726586773191,
 'Caucasian_Woman': 4.149933065595716,
 'Indian_Man': 0.5183585313174947,
 'Indian_Woman': 2.335766423357664}

In [10]:
far

{'African_Man': 0.06802721088435375,
 'African_Woman': 0.0,
 'Asian_Man': 0.28155795401220085,
 'Asian_Woman': 0.6904487917146145,
 'Caucasian_Man': 0.3108348134991119,
 'Caucasian_Woman': 0.9358288770053476,
 'Indian_Man': 0.3452740612861459,
 'Indian_Woman': 0.4398826979472141}

In [11]:
scores

{'African_Man': 50.025514543289674,
 'African_Woman': 49.586776859504134,
 'Asian_Man': 50.29349612585114,
 'Asian_Woman': 50.20103388856979,
 'Caucasian_Man': 50.477247502774695,
 'Caucasian_Woman': 51.63879598662207,
 'Indian_Man': 50.107944732297064,
 'Indian_Woman': 50.841258229700074}